### Production

In [2]:
import mlflow
import pyarrow.feather as feather


In [3]:
# Obtain information
X_train = feather.read_feather("./results/X_train.ftr")
X_test = feather.read_feather("./results/X_test.ftr")
y_train = feather.read_feather("./results/y_train.ftr").to_numpy().ravel()
y_test = feather.read_feather("./results/y_test.ftr").to_numpy().ravel()

In [32]:
# Select best model
experiment_id = "3"
df = mlflow.search_runs(experiment_ids=[experiment_id])
run_id = df.loc[df['metrics.f1-score'].idxmax()]['run_id']
run_name = df.loc[df['metrics.f1-score'].idxmax()]['tags.mlflow.runName']
df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.auc,metrics.recall,metrics.f1-score,metrics.precision,...,params.min_samples_split,params.min_samples_leaf,params.kernel,params.base_estimator,params.bootstrap_features,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name
0,7dc2003696754289aeead34c5bfac44a,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 11:33:05.967000+00:00,2022-04-21 11:38:46.405000+00:00,0.928091,0.928434,0.928484,0.928601,...,None,None,None,None,None,"[{""run_id"": ""7dc2003696754289aeead34c5bfac44a""...",model_ad,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...
1,098c16a028c74f7ebb88f833c080a2c1,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 11:32:37.659000+00:00,2022-04-21 11:33:05.866000+00:00,0.889578,0.888666,0.888905,0.889925,...,None,None,None,None,None,"[{""run_id"": ""098c16a028c74f7ebb88f833c080a2c1""...",model_dt,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...
2,0c3abab9cd9644b6b55403b80e456c4d,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 11:32:16.400000+00:00,2022-04-21 11:32:37.480000+00:00,0.859699,0.857946,0.858354,0.860466,...,None,None,None,None,None,"[{""run_id"": ""0c3abab9cd9644b6b55403b80e456c4d""...",model_lr,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...
3,4629ffe501794896b790b24c57c782f6,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 11:31:40.041000+00:00,2022-04-21 11:32:16.328000+00:00,0.774050,0.767516,0.768289,0.779178,...,None,None,None,None,None,"[{""run_id"": ""4629ffe501794896b790b24c57c782f6""...",model_km,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...
4,7c1fd207aefa42219046d37ac24a6833,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 11:26:42.078000+00:00,2022-04-21 11:26:59.265000+00:00,0.958237,0.959347,0.959326,0.959337,...,None,None,None,None,None,"[{""run_id"": ""7c1fd207aefa42219046d37ac24a6833""...",model_nn,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...
5,9f9836f9a2894b87a7d52b72997679d6,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 02:37:31.239000+00:00,2022-04-21 04:46:40.819000+00:00,0.961509,0.963351,0.963299,0.963460,...,2,1,None,None,None,"[{""run_id"": ""9f9836f9a2894b87a7d52b72997679d6""...",model_rf,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...
6,05e72c28ed854cad88b742a98d742ffc,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 01:54:30.959000+00:00,2022-04-21 02:37:31.041000+00:00,0.954806,0.955305,0.955313,0.955324,...,None,None,rbf,None,None,"[{""run_id"": ""05e72c28ed854cad88b742a98d742ffc""...",model_svm,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...
7,c692c9e94d53476fa3431ed1dedbbe91,3,FINISHED,file:///C:/Users/Laura/Documents/Gonzalo/CEIA/...,2022-04-21 01:33:17.985000+00:00,2022-04-21 01:48:26.336000+00:00,0.958814,0.960733,0.960676,0.960844,...,None,None,None,DecisionTreeClassifier(),False,"[{""run_id"": ""c692c9e94d53476fa3431ed1dedbbe91""...",model_bg,Laura,LOCAL,C:\Users\Laura\anaconda3\lib\site-packages\ipy...


In [24]:
# Test best model
logged_model = f"runs:/{run_id}/{run_name}"

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))



array([1, 1, 0, ..., 0, 1, 0])

In [34]:
# Run server with best model
!mlflow models serve -m ./mlruns/$experiment_id/$run_id/artifacts/$run_name -p 1234

^C


In [ ]:
# mlflow models serve -m ./mlruns/0/8a0ad36647ea44cf97c2c40eb4ca3bf1/artifacts/model -p 1234
# curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["Customer Type", "Age", "Type of Travel", "Class", "Flight Distance", "Inflight wifi service", "Ease of Online booking", "Gate location", "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Arrival Delay in Minutes"],"data":[[0.47603968,  0.81788702,  0.66337845, -1.07706454, -1.04179459, 1.70385282, 0.17214266,  0.79783121, -0.1617392 , 0.54469211, -0.34030327, 1.22713822, 1.25917014, 1.25097391, 1.16171635, -1.03534761, 1.14406253, 1.29899809, 1.94849759]]}' http://127.0.0.1:1234/invocations